In [2]:
#USED: September 29, 2020
from sklearn.model_selection import train_test_split as TTS, cross_validate as cv 
from sklearn.preprocessing import MinMaxScaler as MMS, StandardScaler as SScaler
from sklearn.metrics import mean_squared_error as MSE, r2_score as r2
from matplotlib import pyplot as plt
import time, scipy, numpy as np, math, pickle, random, warnings, pandas as pd, csv, hyperas
from sklearn.pipeline import Pipeline
from hyperopt import STATUS_OK, tpe, Trials, hp, fmin
from functools import partial
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.python.keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.python.framework import ops
import flair
from joblib import dump, load
import warnings
warnings.filterwarnings("ignore")
#'Ex1' : hp.uniform('Ex1', -8, 4),
#'Ex2' : hp.uniform('Ex2', -25, -5),
#'Ex3' : hp.uniform('Ex3', -6, 3),
#


    #This notebook contains algorithm for searching for io values that fit data from current and FE data
    #derived YOGI


#
#Last Used Nov 1
#Fundamentals of charge transfer on an electrode: JACS, 2005, 127, 7815-7824; JACS, 2005, 127, 13949-13954; J. Phys. Chem. B, 2006, 110, 19433-19442 (optional)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
#declare your global variables here
name = 'simple'
scale = load(f'./Non_Yogi_Models/{name}_Model/scale_{name}.bin')
i_model = tf.keras.models.load_model(f'./Non_Yogi_Models/{name}_Model/planar_i_{name}')
fe_model = tf.keras.models.load_model(f'./Non_Yogi_Models/{name}_Model/planar_FE_{name}')
def gen_search_space(num_orders):
    mod_ss = {
        'i1c' : hp.uniform('i1c', -15, 0),
        'i1h' : hp.uniform('i1h', -17.3, 0),
        'i2h' : hp.uniform('i2h', -25, 0),
        'i2c' : hp.uniform('i2c', -15, 0),
        'dif' : hp.quniform('dif', 30, 350,1),
    }
       # 'i2h' : hp.uniform('i2h', -25, 0),
       # 'i2c' : hp.uniform('i2c', -15, 0),
       # 'Ex1' : hp.uniform('Ex1', -10.5, 2),
       # 'Ex2' : hp.uniform('Ex2', -20, 0),
       # 'Ex3' : hp.uniform('Ex3', -20, 4),
    for i in range(num_orders):
        key = 'ord' + str(i+1)
        mod_ss[key] = hp.uniform('o' + str(i+1), 0, 1)
    return mod_ss
lst = [[-0.53,-5.08498,-7.093,0, 0.1, 0.984]] #CHANGE
lst = scale.transform(lst)
print(fe_model.predict(lst))
print(5*(0.79-0.00819)/0.00819)

[[0.92763615]]
477.2954822954824


In [4]:
'''
def fix_indices(ar, ar2):
    #parameters passed in wrong order... annoying swap necessary... dicts are not inherently ordered
    ar2[0][0] = 0
    ar2[0][1] = 0
    ar2[0][2] = ar.get('Ex1')
    ar2[0][3] = ar.get('Ex2')
    ar2[0][4] = ar.get('Ex3') 
    ar2[0][5] = ar.get('dif')
    return ar2
'''

def fix_indices(ar, ar2):
    #parameters passed in wrong order... annoying swap necessary... dicts are not inherently ordered
    ar2[0][0] = 0
    ar2[0][1] = ar.get('i1c')
    ar2[0][2] = ar.get('i1h')
    ar2[0][3] = ar.get('dif')
    ar2[0][4] = 0
    ar2[0][5] = 0
    return ar2

In [5]:
def adj_potentials(potentials, ords):
    for i in range(len(potentials)):
        ar = np.array([potentials[i],0,0,0,0,ords[i]]) #CHANGE
        #ar = np.array([potentials[i],ords[i],0,0,0,0]) #CHANGE
        ar = ar.reshape(1,-1)
        ar = scale.transform(ar)
        potentials[i] = ar[0][0]
        ords[i] = ar[0][5] #CHANGE
        #ords[i] = ar[0][1] #CHANGE
    return potentials, ords

def hypertune_BO(params, pots, fes, curs):
    lst = list(range(6)) #CHANGE
    ar2 = np.array(lst, ndmin=2, dtype=float)
    #parameters passed in wrong order... annoying swap necessary... dicts are not inherently ordered
    ar2 = fix_indices(params, ar2)
    ar2 = scale.transform(ar2) 

    pred_i_totals = np.zeros(len(curs)) #empty array for predicted currents
    pred_fes = np.zeros(len(fes)) #empty array for predicted FE
    true_is = curs
    true_fes = fes
    #print(true_is, true_fes, pots)
    ords = [params.get('ord' + str(i+1)) for i in range(len(pots))]
    #print('Entering Ords')
    _, ords = adj_potentials(np.zeros(len(pots)) ,ords)

    for i in range(len(pots)):
        ar2[0][0] = pots[i]
        ar2[0][5] = ords[i] #CHANGE
        pred_i_totals[i] = i_model.predict(ar2)
        pred_fes[i] = fe_model.predict(ar2)

    loss_lst = []
    for i in range(len(pred_i_totals)): loss_lst.append(abs((pred_i_totals[i] - true_is[i])/true_is[i]))
    for i in range(len(pred_fes)): loss_lst.append(5*abs((pred_fes[i] - true_fes[i])/true_fes[i]))
    #print(loss_lst)
    loss = sum(loss_lst)
    max_err = max(loss_lst)
    
    return {'loss':max_err, 'params':params, 'max_loss':max_err, 'status':STATUS_OK} #'max_loss':max_err

In [6]:
def run_trials(data, pots, file_name, mod_ss):

    trials_step = 10  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 10  # initial max_trials. put something small to not have to wait
    #print(f'Trials Potentials {pots}')
    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open(file_name, 'rb')) #CHANGE
        max_trials = len(trials.trials) + trials_step
        #print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()
        
    objective = partial(hypertune_BO, pots=pots, fes=data['FE'], curs=data['i']) 
    best = fmin(fn=objective, space=mod_ss, algo=tpe.suggest, max_evals=max_trials, trials=trials)
    #CORRECTED ABOVE
    
    for key in best.keys(): best[key] = round(best.get(key),5)
    print("Best:", best)
    ans_fe = []
    ans_i = []
    for i in range(len(pots)):
        order = 'o' + str(i+1)
        #lst = [[pots[i], best.get(order), best.get('Ex1'), best.get('Ex2'), best.get('Ex3'), best.get('dif')]] #CHANGE
        lst = [[pots[i], best.get('i1c'), best.get('i1h'), best.get('dif'), 0.1, best.get(order)]] #CHANGE
        lst = scale.transform(lst)
        lst[0][0] = pots[i]
        ans_fe.append(fe_model.predict(lst)[0][0])
        ans_i.append(i_model.predict(lst)[0][0])      
    print("FE : " + str(ans_fe))
    #print("Real FE: " + str(data['FE']))
    print("Cur : " + str(ans_i))
    #print("Real Cur: " + str(data['i']))
    # save the trials object
    with open(file_name, 'wb') as f: #CHANGE
        pickle.dump(trials, f)
    return trials, best

# loop indefinitely and stop whenever you like
#need to give the true vals to trials
def BO_Search(data, pots, file_name, mod_ss):
    i = 0
    iters = 300
    #print(f'BO_Search Potentials {pots}')
    pots, _ = adj_potentials(data['E'], np.zeros(len(data)))
    while (i < iters): 
        trials, best = run_trials(data, pots, file_name, mod_ss)
        i+=1
    return best

In [8]:
data = pd.read_csv('gold_lit_data_Dec3.csv')
results = []
for source in data['source'].unique():
    print(source)
    cur = data[data['source'] == source].copy()
    if (source != 'Jingyu'): continue
    mod_ss = gen_search_space(len(cur))
    file_name = f'./{name}_{source}.hyperopt'
    og_pots = cur['E'].copy()
    best = BO_Search(cur, og_pots, file_name, mod_ss)
    print('\n\n\n\n\n' + file_name + ' ' + str(best) + '\n\n\n\n\n\n') 
    results.append(best)

Jingyu
100%|██████████| 10/10 [00:03<00:00,  2.84trial/s, best loss: 3.2842279572283615]
Best: {'dif': 56.0, 'i1c': -4.51151, 'i1h': -6.76699, 'o1': 0.40607, 'o2': 0.74631, 'o3': 0.80959, 'o4': 0.7091, 'o5': 0.15289, 'o6': 0.10632}
FE : [0.9563276, 0.966861, 0.96109605, 0.939189, 0.9004972, 0.7024003]
Cur : [-0.9411765, -0.1456919, 0.619731, 1.486138, 2.0138543, 2.6031244]
100%|██████████| 20/20 [00:03<00:00,  2.65trial/s, best loss: 3.2842279572283615]
Best: {'dif': 56.0, 'i1c': -4.51151, 'i1h': -6.76699, 'o1': 0.40607, 'o2': 0.74631, 'o3': 0.80959, 'o4': 0.7091, 'o5': 0.15289, 'o6': 0.10632}
FE : [0.9563276, 0.966861, 0.96109605, 0.939189, 0.9004972, 0.7024003]
Cur : [-0.9411765, -0.1456919, 0.619731, 1.486138, 2.0138543, 2.6031244]
100%|██████████| 30/30 [00:03<00:00,  2.78trial/s, best loss: 2.0584621800644785]
Best: {'dif': 323.0, 'i1c': -3.53853, 'i1h': -7.0701, 'o1': 0.5439, 'o2': 0.76019, 'o3': 0.15809, 'o4': 0.76321, 'o5': 0.61803, 'o6': 0.43867}
FE : [1.1163981, 1.0983396, 1.

100%|██████████| 230/230 [00:03<00:00,  2.68trial/s, best loss: 1.9090325145704563]
Best: {'dif': 69.0, 'i1c': -2.98883, 'i1h': -6.15431, 'o1': 0.2029, 'o2': 0.82427, 'o3': 0.03454, 'o4': 0.17638, 'o5': 0.9173, 'o6': 0.44254}
FE : [1.0929561, 1.079855, 1.044116, 0.9681437, 0.9165972, 0.5136823]
Cur : [0.43094853, 1.205327, 1.7565993, 2.2953575, 2.5084996, 2.9016628]
100%|██████████| 240/240 [00:04<00:00,  2.45trial/s, best loss: 1.9090325145704563]
Best: {'dif': 69.0, 'i1c': -2.98883, 'i1h': -6.15431, 'o1': 0.2029, 'o2': 0.82427, 'o3': 0.03454, 'o4': 0.17638, 'o5': 0.9173, 'o6': 0.44254}
FE : [1.0929561, 1.079855, 1.044116, 0.9681437, 0.9165972, 0.5136823]
Cur : [0.43094853, 1.205327, 1.7565993, 2.2953575, 2.5084996, 2.9016628]
100%|██████████| 250/250 [00:03<00:00,  2.72trial/s, best loss: 1.9090325145704563]
Best: {'dif': 69.0, 'i1c': -2.98883, 'i1h': -6.15431, 'o1': 0.2029, 'o2': 0.82427, 'o3': 0.03454, 'o4': 0.17638, 'o5': 0.9173, 'o6': 0.44254}
FE : [1.0929561, 1.079855, 1.044116,

100%|██████████| 450/450 [00:03<00:00,  2.51trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827, 0.9779313, 0.95277977, 0.9158803, 0.7933209, 0.48559368]
Cur : [-0.05036137, 0.69499016, 1.430896, 2.1458566, 2.5615337, 2.8972704]
100%|██████████| 460/460 [00:03<00:00,  2.64trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827, 0.9779313, 0.95277977, 0.9158803, 0.7933209, 0.48559368]
Cur : [-0.05036137, 0.69499016, 1.430896, 2.1458566, 2.5615337, 2.8972704]
100%|██████████| 470/470 [00:04<00:00,  2.49trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827

100%|██████████| 670/670 [00:03<00:00,  2.56trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827, 0.9779313, 0.95277977, 0.9158803, 0.7933209, 0.48559368]
Cur : [-0.05036137, 0.69499016, 1.430896, 2.1458566, 2.5615337, 2.8972704]
100%|██████████| 680/680 [00:03<00:00,  2.54trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827, 0.9779313, 0.95277977, 0.9158803, 0.7933209, 0.48559368]
Cur : [-0.05036137, 0.69499016, 1.430896, 2.1458566, 2.5615337, 2.8972704]
100%|██████████| 690/690 [00:04<00:00,  2.32trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827

100%|██████████| 890/890 [00:04<00:00,  2.46trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827, 0.9779313, 0.95277977, 0.9158803, 0.7933209, 0.48559368]
Cur : [-0.05036137, 0.69499016, 1.430896, 2.1458566, 2.5615337, 2.8972704]
100%|██████████| 900/900 [00:04<00:00,  2.34trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827, 0.9779313, 0.95277977, 0.9158803, 0.7933209, 0.48559368]
Cur : [-0.05036137, 0.69499016, 1.430896, 2.1458566, 2.5615337, 2.8972704]
100%|██████████| 910/910 [00:04<00:00,  2.22trial/s, best loss: 1.3062075732195106]
Best: {'dif': 54.0, 'i1c': -3.58845, 'i1h': -6.05233, 'o1': 0.65324, 'o2': 0.33709, 'o3': 0.20198, 'o4': 0.18329, 'o5': 0.05072, 'o6': 0.41308}
FE : [0.99823827

...

100%|██████████| 1980/1980 [00:04<00:00,  2.02trial/s, best loss: 1.2313514578435136]
Best: {'dif': 70.0, 'i1c': -3.68291, 'i1h': -6.05135, 'o1': 0.79677, 'o2': 0.43056, 'o3': 0.57522, 'o4': 0.11443, 'o5': 0.54127, 'o6': 0.36206}
FE : [0.98497766, 0.9650017, 0.94581383, 0.89865977, 0.8188655, 0.4344296]
Cur : [-0.09181303, 0.6081891, 1.344824, 2.0679655, 2.4088838, 2.8458633]
100%|██████████| 1990/1990 [00:04<00:00,  2.02trial/s, best loss: 1.2313514578435136]
Best: {'dif': 70.0, 'i1c': -3.68291, 'i1h': -6.05135, 'o1': 0.79677, 'o2': 0.43056, 'o3': 0.57522, 'o4': 0.11443, 'o5': 0.54127, 'o6': 0.36206}
FE : [0.98497766, 0.9650017, 0.94581383, 0.89865977, 0.8188655, 0.4344296]
Cur : [-0.09181303, 0.6081891, 1.344824, 2.0679655, 2.4088838, 2.8458633]
100%|██████████| 2000/2000 [00:04<00:00,  2.04trial/s, best loss: 1.2313514578435136]
Best: {'dif': 70.0, 'i1c': -3.68291, 'i1h': -6.05135, 'o1': 0.79677, 'o2': 0.43056, 'o3': 0.57522, 'o4': 0.11443, 'o5': 0.54127, 'o6': 0.36206}
FE : [0.9849

100%|██████████| 2200/2200 [00:04<00:00,  2.08trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2210/2210 [00:04<00:00,  2.09trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2220/2220 [00:04<00:00,  2.00trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 

100%|██████████| 2420/2420 [00:04<00:00,  2.03trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2430/2430 [00:05<00:00,  1.97trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2440/2440 [00:04<00:00,  2.04trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 

100%|██████████| 2640/2640 [00:05<00:00,  1.86trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2650/2650 [00:05<00:00,  1.83trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2660/2660 [00:05<00:00,  1.82trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 

100%|██████████| 2860/2860 [00:05<00:00,  1.93trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2870/2870 [00:05<00:00,  1.94trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 0.96924436, 0.9562306, 0.90725625, 0.8137328, 0.4418134]
Cur : [0.093917534, 0.8459256, 1.5775906, 2.284172, 2.6158977, 3.0244095]
100%|██████████| 2880/2880 [00:05<00:00,  1.89trial/s, best loss: 1.214125439925756]
Best: {'dif': 37.0, 'i1c': -3.43851, 'i1h': -5.86887, 'o1': 0.37683, 'o2': 0.3197, 'o3': 0.59397, 'o4': 0.18515, 'o5': 0.52387, 'o6': 0.43372}
FE : [0.98403484, 

In [9]:
with open(f'{name}_Best.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
t = pickle.load(open('ogHERandCER_Best.pkl', 'rb'))
t

In [ ]:
for i in range(len(data)):
    row = data.loc[i]
    old_source = source
    if(row['source'] != source):
        print(file_name)
        mod_ss = gen_search_space(len(pots))
        source = row['source']
        pots = [row['E']]
        fes = [round(row['FE'],3)]
        curs = [round(row['i'],3)]
        file_name = 'Dec3_' + str(row['source']) + '.hyperopt'
        #best = BO_Search(pots, pots.copy(), fes, curs, file_name, mod_ss)
        #print('\n\n\n\n\n' + file_name + ' ' + str(best) + '\n\n\n\n\n\n') 
        #results.append(best)
    else:
        pots.append(row['E'])
        fes.append(round(row['FE'],3))
        curs.append(round(row['i'],3))
    i += 1
#with open('./ogHERandCER_Model/BO_results_Dec20.pkl', 'wb') as f: #CHANGE
#    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./Sep15/BO_Results.pkl', 'wb') as f: #CHANGE
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('./Sep15/BO_Results.pkl', 'rb') as f:
    t = pick.load(f)

In [ ]:
objective = partial(hypertune_BO, model=key, scale=True) 
#max_cell_loss = 10000
#min_lss = 10000
#min_ind = 0
#end_iter_lss = 10000
#if (i >= iters - 1):
    #    results = trials.results[min_ind:]
    #   for j in range(len(trials.results) - min_ind):
    #        ind = j + min_ind
    #        if (ind >= len(trials.results)): break
    #        if ((trials.results[ind].get('max_loss') < max_cell_loss)): #& (trials.results[ind].get('loss') <= end_iter_lss)):
    #            min_lss = trials.results[ind].get('loss')
    #            max_cell_loss = trials.results[ind].get('max_loss')
    #            min_ind = ind
    #            if (i == iters - 1): end_iter_lss = min_lss
    #    print('Min Loss: ' + str(min_lss) + '\n' + 'Max Cell Loss: ' + str(max_cell_loss) + '\n' + 'End iter loss: ' + str(end_iter_lss) + '\n' + str(i) + '\n'')

In [ ]:
mod_ss = {
    'i1c' : hp.uniform('i1c', -8.5, 4),
    'i1h' : hp.uniform('i1h', -25, -5),
    'i2h' : hp.uniform('i2h', -6, 3),
    'dif' : hp.quniform('dif', 30, 350,1),
    'ord1' : hp.uniform('o1', 0, 1),
    'ord2' : hp.uniform('o2', 0, 1),
    'ord3' : hp.uniform('o3', 0, 1),
}



In [ ]:
ords = {}

for i in range(4):
    key = 'ord' + str(i+1)
    ords[key] = hp.uniform('o' + str(i+1), 0, 1)
print(ords)

In [ ]:
dic = {'o1':1, 'o2':2, 'o3':3}
ords = [dic.get('o' + str(i+1)) for i in range(3)]
ords


In [ ]:
hel = [0,1,2]
#hev = hel.copy()

print(hev)

In [ ]:
Yogi
        'Ex1' : hp.uniform('Ex1', -10.5, 2),
        'Ex2' : hp.uniform('Ex2', -20, 0),
        'Ex3' : hp.uniform('Ex3', -20, 4),
        'dif' : hp.quniform('dif', 100, 400,1), #old max =350
    
Simple
'i1c' : hp.uniform('i1c', -15, 0),
'i1h' : hp.uniform('i1h', -17.3, -0.74),
'dif' : hp.quniform('dif', 30, 350,1),
    
og_HER
        'i1c' : hp.uniform('i1c', -15, 0),
        'i1h' : hp.uniform('i1h', -17.3, 0),
        'i2h' : hp.uniform('i2h', -25, 0),
        'dif' : hp.quniform('dif', 30, 350,1),

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

